In [ ]:
import pandas as pd
from google.colab import drive
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pickle
import numpy as np
from urllib.parse import quote
import requests

In [ ]:
#data preproccessing
drive.mount('/content/drive')

In [ ]:
file_path = '/content/drive/MyDrive/sensorParser.csv'
df = pd.read_csv(file_path)

In [ ]:
#Realtime
url = "http://103.200.22.212:60000/query?sql=SELECT * FROM sensorParser"
encoded_url = quote(url, safe=':/?=&')
response = requests.get(encoded_url)
data = response.json()
#sử dụng thư viện requests để thực hiện yêu cầu HTTP và lấy dữ liệu từ URL đó. 
#Sau đó, chuyển đổi dữ liệu từ định dạng JSON thành mảng dữ liệu Python
df = pd.DataFrame(data)

In [ ]:
df

,id,id_wasp,id_secret,frame_type,frame_number,sensor,value,timestamp,sync,raw,parser_type,MeshliumID
0,21,SAP_NHOM15,48581CE819623C8B,134,0,PRES,100736.93,2023-05-26 02:20:25,0,noraw,1,20274081525527
1,22,SAP_NHOM15,48581CE819623C8B,134,0,HUM,55.5,2023-05-26 02:20:25,0,noraw,1,20274081525527
2,23,SAP_NHOM15,48581CE819623C8B,134,0,TC,24.53,2023-05-26 02:20:25,0,noraw,1,20274081525527
3,24,SAP_NHOM15,48581CE819623C8B,134,0,BAT,15,2023-05-26 02:20:25,0,noraw,1,20274081525527
4,25,SAP_NHOM15,48581CE819623C8B,134,0,STR,new_sensor_frame,2023-05-26 02:20:25,0,noraw,1,20274081525527
...,...,...,...,...,...,...,...,...,...,...,...,...
1200,1270,SAP_NHOM15,48581CE819623C8B,134,5,PRES,100709.67,2023-05-26 02:58:01,0,noraw,1,20274081525527
1201,1271,SAP_NHOM15,48581CE819623C8B,134,5,HUM,56.2,2023-05-26 02:58:01,0,noraw,1,20274081525527
1202,1272,SAP_NHOM15,48581CE819623C8B,134,5,TC,24.54,2023-05-26 02:58:01,0,noraw,1,20274081525527
1203,1273,SAP_NHOM15,48581CE819623C8B,134,5,BAT,19,2023-05-26 02:58:01,0,noraw,1,20274081525527


In [ ]:
cols_to_keep = ['sensor', 'value', 'timestamp']  # Các cột chỉ định giữ lại

df = df[cols_to_keep]  # Giữ lại các cột chỉ định

df.head(10)

,sensor,value,timestamp
0,PRES,100736.93,2023-05-26 02:20:25
1,HUM,55.5,2023-05-26 02:20:25
2,TC,24.53,2023-05-26 02:20:25
3,BAT,15,2023-05-26 02:20:25
4,STR,new_sensor_frame,2023-05-26 02:20:25
5,PRES,100740.69,2023-05-26 02:20:05
6,HUM,55.1,2023-05-26 02:20:05
7,TC,24.52,2023-05-26 02:20:05
8,BAT,15,2023-05-26 02:20:05
9,STR,new_sensor_frame,2023-05-26 02:20:05


In [ ]:
df = df[~df['sensor'].isin(['BAT', 'STR'])]
#Bỏ các hàng trong df mà cột "sensor" có giá trị là BAT hoặc STR. 
#"~" lấy các hàng không thoả điều kiện này
df.head(10)

,sensor,value,timestamp
0,PRES,100736.93,2023-05-26 02:20:25
1,HUM,55.5,2023-05-26 02:20:25
2,TC,24.53,2023-05-26 02:20:25
5,PRES,100740.69,2023-05-26 02:20:05
6,HUM,55.1,2023-05-26 02:20:05
7,TC,24.52,2023-05-26 02:20:05
10,PRES,100735.05,2023-05-26 02:19:42
11,HUM,55.1,2023-05-26 02:19:42
12,TC,24.54,2023-05-26 02:19:42
15,PRES,100737.53,2023-05-26 02:19:25


In [ ]:
# Tách thành các thuộc tính mới A, B, C
df_new = df.pivot(index='timestamp', columns='sensor', values='value')
df_new = df_new.sort_values(by='timestamp').reset_index()

In [ ]:
df_new.head(10)

sensor,timestamp,HUM,PRES,TC
0,2023-05-19 03:05:38,57.1,100587.03,24.49
1,2023-05-19 03:06:01,57.3,100582.67,24.47
2,2023-05-19 03:06:18,57.2,100583.13,24.46
3,2023-05-19 03:06:35,57.1,100586.05,24.44
4,2023-05-19 03:06:52,57.1,100581.08,24.41
5,2023-05-19 03:07:15,57.1,100580.70,24.37
6,2023-05-19 03:07:32,58.3,100579.50,24.33
7,2023-05-19 03:07:49,58.7,100585.33,24.30
8,2023-05-19 03:08:12,59.3,100579.74,24.25
9,2023-05-19 03:08:29,59.3,100574.34,24.24


In [ ]:
df = df_new.drop('HUM', axis=1)

In [ ]:
df = df.drop('PRES', axis=1)

In [ ]:
df

sensor,timestamp,TC
0,2023-05-19 03:05:38,24.49
1,2023-05-19 03:06:01,24.47
2,2023-05-19 03:06:18,24.46
3,2023-05-19 03:06:35,24.44
4,2023-05-19 03:06:52,24.41
...,...,...
236,2023-05-26 02:58:41,24.60
237,2023-05-26 02:58:58,24.61
238,2023-05-26 02:59:15,24.60
239,2023-05-26 02:59:38,24.59


In [ ]:
# Chuyển cột timestamp thành định dạng datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])

# Trích xuất năm, tháng và ngày từ cột timestamp
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day

# Loại bỏ cột timestamp ban đầu (nếu cần)
df = df.drop('timestamp', axis=1)

In [ ]:
df

sensor,TC,year,month,day
0,24.49,2023,5,19
1,24.47,2023,5,19
2,24.46,2023,5,19
3,24.44,2023,5,19
4,24.41,2023,5,19
...,...,...,...,...
236,24.60,2023,5,26
237,24.61,2023,5,26
238,24.60,2023,5,26
239,24.59,2023,5,26


In [ ]:
# Xử lý dữ liệu
X = df[['year', 'month', 'day']].values
y = df[['TC']].values

In [ ]:
# Chuẩn hóa dữ liệu
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y)

In [ ]:
# Chia dữ liệu thành tập train và tập test
#70,30
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=1)

In [ ]:
# Reshape dữ liệu cho LSTM (số mẫu, số time steps, số đặc trưng)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [ ]:
# Tạo mô hình LSTM
model = Sequential()
model.add(LSTM(2, input_shape=(1, X_train.shape[2])))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0)

In [ ]:
model.save('model.h5')


In [ ]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
#đánh giá mô hình
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error (MSE):", mse)

3/3 [==============================] - 1s 6ms/step - loss: 0.1575
Mean Squared Error (MSE): 0.15748614072799683


In [ ]:
#Test
X = np.array([[2023, 5, 30]])
# Reshape input features for LSTM
X_reshaped = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))

# Make prediction
prediction = model.predict(X_reshaped)

# Create a new scaler for target variable
target_scaler = MinMaxScaler()

# Fit target variable to new scaler
target_scaler.fit(y.reshape(-1, 1))

# Inverse transform the prediction to the original scale
prediction_rescaled = target_scaler.inverse_transform(prediction)

# Print the predicted temperature for the new date
new_date_index = 0  # Thay đổi chỉ mục tương ứng với bộ ngày tháng mới cần dự đoán
print("Predicted Temperature:", prediction_rescaled[new_date_index][0])

8/8 [==============================] - 0s 2ms/step
Predicted Temperature: 24.419958


In [ ]:
# Xuất model thành file pickle
pickle.dump(model, open('model.pkl', 'wb'))